In [133]:
import random

class Sentiment:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"
    NEUTRAL = "NEUTRAL"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #score is 4 or 5
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
    
    def evenly_distributed(self):
        negative = list(filter(lambda x : x.sentiment == Sentiment.NEGATIVE, self.reviews )) 
        positive = list(filter(lambda x : x.sentiment == Sentiment.POSITIVE, self.reviews ))    
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative  + positive_shrunk
        random.shuffle(self.reviews)
    
       
        
        

In [87]:
import json
file_name = './sentiment/books_small_10000.json'
reviews = []
with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))
        
reviews[5].text        

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [88]:
from sklearn.model_selection import train_test_split

In [140]:
training , test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)

cont.evenly_distributed()

len(cont.reviews)

872

In [157]:
train_container.evenly_distributed()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distributed()
test_x = train_container.get_text()
test_y = train_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


# Bag of Words Vectorization


In [180]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

#This book is great
#This book was so bad

count_vect = TfidfVectorizer()
train_x_count = count_vect.fit_transform(train_x) 

test_x_count = count_vect.transform(test_x)
print(train_x[12])
print(train_x_count[12].toarray())

Too bogged down with details that don't matter.  Not his best work.  But..... It it's a good story, with a great ending.
[[0. 0. 0. ... 0. 0. 0.]]


# CLASSIFICATION


In [174]:
from sklearn import svm

clf_svm = svm.SVC(kernel = 'linear')

In [181]:
clf_svm.fit(train_x_count, train_y)

test_x[0]

clf_svm.predict(test_x_count[0])

array(['NEGATIVE'], dtype='<U8')

#### Decision Tree

In [182]:
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_count, train_y)

clf_dec.predict(test_x_count[0])

array(['NEGATIVE'], dtype='<U8')

#### Naive Bayes

In [164]:
from sklearn.naive_bayes import GaussianNB
#clf_gnb = GaussianNB()
#clf_gnb.fit(train_x_count, train_y).predict(test_X_count[0])



#### Logistic Tree

In [183]:
from sklearn.linear_model import LogisticRegression

clf_lr = LogisticRegression()
clf_lr.fit(train_x_count, train_y)

clf_lr.predict(test_x_count[0])

array(['NEGATIVE'], dtype='<U8')

# Evaluation

In [184]:
# Mean Accuracy
print(clf_svm.score(test_x_count, test_y))
print(clf_dec.score(test_x_count, test_y))
print(clf_lr.score(test_x_count, test_y))

0.9885321100917431
1.0
0.9655963302752294


In [185]:
#F1 Scores
from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_x_count) , average= None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])
#f1_score(test_y, clf_lr.predict(test_x_count) , average= None, labels=[Sentiment.POSITIVE, Sentiment.NEUTRAL, Sentiment.NEGATIVE])

C:\Users\rahul\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


array([0.98850575, 0.        , 0.98855835])

In [186]:
test_set = ['great for my wedding', "loved it in my garden", 'good computer']
new_test = count_vect.transform(test_set)

clf_svm.predict(new_test)

array(['POSITIVE', 'POSITIVE', 'POSITIVE'], dtype='<U8')

# Grid Search Cv Model

In [193]:
from sklearn.model_selection import GridSearchCV


parameters = {'kernel': ('linear', 'rbf'), 'C':(1,4,6,8,16,32)}

svc = svm.SVC()
clf  =GridSearchCV(svc, parameters, cv= 5)
clf.fit(train_x_count, train_y)



GridSearchCV(cv=5, error_score=nan,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=None,
             param_grid={'C': (1, 4, 6, 8, 16, 32),
                         'kernel': ('linear', 'rbf')},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [194]:
print(clf.score(test_x_count, test_y))

1.0



#  Saving The Model

In [203]:
import pickle

with open('./sentiment/Sentiment.pkl', 'wb') as f:
    pickle.dump(clf, f)
with open('./sentiment/Sentiment.pkl', 'rb') as f:
    loaded_clf = pickle.load(f)    

Load model

In [205]:
print(test_x[0])

loaded_clf.predict(test_x_count[0])
      

though interesting, I found it difficult to keep interested because of the diction.  If it was truly written in the 1800s I can understand why.


array(['NEGATIVE'], dtype='<U8')